# Web Scraping peleadores y estadisticas

## Import libraries

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as BS
import requests
import json
import time
import unicodedata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn

### pruebas

In [2]:
df = pd.read_html('https://es.wikipedia.org/wiki/Anexo:Eventos_de_UFC')

In [3]:
eventos = df[0].copy()

In [4]:
eventos.shape

(599, 6)

In [5]:
eventos.tail(20)

,#,Evento,Fecha,Sede,Localización,Fuente
579,571,UFC on ESPN: Sandhagen vs. Dillashaw,24 de julio de 2021,UFC Apex,"Las Vegas, Nevada",NaN
580,572,UFC on ESPN: Hall vs. Strickland,31 de julio de 2021,UFC Apex,"Las Vegas, Nevada",NaN
581,573,UFC 265: Lewis vs. Gane,7 de agosto de 2021,Toyota Center,"Houston, Texas",NaN
582,574,UFC on ESPN: Cannonier vs. Gastelum,21 de agosto de 2021,UFC Apex,"Las Vegas, Nevada",NaN
583,575,UFC on ESPN: Barboza vs. Chikadze,28 de agosto de 2021,UFC Apex,"Las Vegas, Nevada",NaN
584,576,UFC Fight Night: Brunson vs. Till,4 de septiembre de 2021,UFC Apex,"Las Vegas, Nevada",NaN
585,577,UFC Fight Night: Smith vs. Spann,18 de septiembre de 2021,UFC Apex,"Las Vegas, Nevada",NaN
586,578,UFC 266: Volkanovski vs. Ortega,25 de septiembre de 2021,T-Mobile Arena,"Paradise, Nevada",NaN
587,579,UFC Fight Night: Santos vs. Walker,2 de octubre de 2021,UFC Apex,"Las Vegas, Nevada",NaN
588,580,UFC Fight Night: Dern vs. Rodriguez,9 de octubre de 2021,UFC Apex,"Las Vegas, Nevada",NaN


In [6]:
eventos['Fuente'] = eventos['Fuente'].fillna('-')
NO_eventos = eventos[eventos['Fuente'].str.contains('Cancelado')].index
eventos.drop(NO_eventos, inplace = True)
eventos.shape

(590, 6)

In [7]:
eventos.drop(['#', 'Fuente'], axis = 1, inplace = True)
eventos.reset_index(drop=True, inplace = True)

In [8]:
eventos.index

RangeIndex(start=0, stop=590, step=1)

In [9]:
response = requests.get('https://es.wikipedia.org/wiki/Anexo:Eventos_de_UFC')
soup = BS(response.text)

In [10]:
wiki_baselink = 'https://es.wikipedia.org'
scrap_table = soup.find('tbody')
rows = scrap_table.find_all('tr')[1:]
wikilink_event = []
for row in rows:
    event_link = row.find_all('a')[0]['href']
    complete_link = wiki_baselink + event_link
    wikilink_event.append(complete_link)

In [11]:
eventos['link_wiki'] = pd.Series(wikilink_event)

In [12]:
eventos.tail()

,Evento,Fecha,Sede,Localización,link_wiki
585,UFC Fight Night: Vieira vs. Tate,20 de noviembre de 2021,UFC Apex,"Las Vegas, Nevada",https://es.wikipedia.org/wiki/UFC_Fight_Night:...
586,UFC on ESPN: Font vs. Aldo,4 de diciembre de 2021,UFC Apex,"Las Vegas, Nevada",https://es.wikipedia.org/wiki/UFC_266
587,UFC 269: Oliveira vs. Poirier,11 de diciembre de 2021,T-Mobile Arena,"Las Vegas, Nevada",https://es.wikipedia.org/wiki/UFC_Fight_Night:...
588,UFC Fight Night: Lewis vs. Daukaus,18 de diciembre de 2021,UFC Apex,"Las Vegas, Nevada",https://es.wikipedia.org/wiki/UFC_Fight_Night:...
589,UFC on ESPN: Kattar vs. Chikadze,15 de enero de 2022,UFC Apex,"Las Vegas, Nevada",https://es.wikipedia.org/wiki/UFC_Fight_Night:...


In [13]:
eventos.shape

(590, 5)

## Recaudacion eventos scraping

In [14]:
ingresos = []
for link in eventos['link_wiki']:
    response = requests.get(link)
    soup = BS(response.text)
    scrap_table = soup.find('tbody')
    rows = scrap_table.find_all('tr')
    elementos = [row.text.split('\n') for row in rows]
    for elemento in elementos:
        if 'Recaudación' in elemento:
            ingresos.append(unicodedata.normalize("NFKD", elemento[1]))
            break
        if elemento == elementos[-1]:
            ingresos.append('Not Found')

len(ingresos) 

590

In [16]:
eventos['ingresos_totales'] = pd.Series(ingresos)
eventos.tail()

,Evento,Fecha,Sede,Localización,link_wiki,ingresos_totales
585,UFC Fight Night: Vieira vs. Tate,20 de noviembre de 2021,UFC Apex,"Las Vegas, Nevada",https://es.wikipedia.org/wiki/UFC_Fight_Night:...,Not Found
586,UFC on ESPN: Font vs. Aldo,4 de diciembre de 2021,UFC Apex,"Las Vegas, Nevada",https://es.wikipedia.org/wiki/UFC_266,"$5,609,906[1]​"
587,UFC 269: Oliveira vs. Poirier,11 de diciembre de 2021,T-Mobile Arena,"Las Vegas, Nevada",https://es.wikipedia.org/wiki/UFC_Fight_Night:...,Not Found
588,UFC Fight Night: Lewis vs. Daukaus,18 de diciembre de 2021,UFC Apex,"Las Vegas, Nevada",https://es.wikipedia.org/wiki/UFC_Fight_Night:...,Not Found
589,UFC on ESPN: Kattar vs. Chikadze,15 de enero de 2022,UFC Apex,"Las Vegas, Nevada",https://es.wikipedia.org/wiki/UFC_Fight_Night:...,Not Found


### Formatear ingresos

In [17]:
def format_ingresos(ingreso_str):
    try:
        formated_ingreso = ingreso_str.split('[')[0].replace('$', '').replace(',', '')
    except:
        formated_ingreso = ingreso_str
    return formated_ingreso

In [31]:
eventos['ingresos_dollar'] = eventos['ingresos_totales'].apply(format_ingresos)
eventos.tail()

,Evento,Fecha,Sede,Localización,link_wiki,ingresos_totales,ingresos_dollar
585,UFC Fight Night: Vieira vs. Tate,20 de noviembre de 2021,UFC Apex,"Las Vegas, Nevada",https://es.wikipedia.org/wiki/UFC_Fight_Night:...,Not Found,Not Found
586,UFC on ESPN: Font vs. Aldo,4 de diciembre de 2021,UFC Apex,"Las Vegas, Nevada",https://es.wikipedia.org/wiki/UFC_266,"$5,609,906[1]​",5609906
587,UFC 269: Oliveira vs. Poirier,11 de diciembre de 2021,T-Mobile Arena,"Las Vegas, Nevada",https://es.wikipedia.org/wiki/UFC_Fight_Night:...,Not Found,Not Found
588,UFC Fight Night: Lewis vs. Daukaus,18 de diciembre de 2021,UFC Apex,"Las Vegas, Nevada",https://es.wikipedia.org/wiki/UFC_Fight_Night:...,Not Found,Not Found
589,UFC on ESPN: Kattar vs. Chikadze,15 de enero de 2022,UFC Apex,"Las Vegas, Nevada",https://es.wikipedia.org/wiki/UFC_Fight_Night:...,Not Found,Not Found


In [34]:
notfound_eventos = eventos[eventos['ingresos_totales'] == 'Not Found']
notfound_eventos


,Evento,Fecha,Sede,Localización,link_wiki,ingresos_totales,ingresos_dollar
14,UFC 13: The Ultimate Force,30 de mayo de 1997,Augusta Civic Center,"Augusta, Georgia, USA",https://es.wikipedia.org/wiki/UFC_13,Not Found,Not Found
15,UFC 14: Showdown,27 de julio de 1997,Boutwell Auditorium,"Birmingham, Alabama, USA",https://es.wikipedia.org/wiki/UFC_14,Not Found,Not Found
16,UFC 15: Collision Course,17 de octubre de 1997,Casino Magic Bay St. Louis,"Bay St. Louis, Mississippi, USA",https://es.wikipedia.org/wiki/UFC_15,Not Found,Not Found
17,UFC Ultimate Japan,21 de diciembre de 1997,Yokohama Arena,"Yokohama, Japón",https://es.wikipedia.org/wiki/UFC_Japan,Not Found,Not Found
18,UFC 16: Battle in the Bayou,13 de marzo de 1998,Pontchartrain Center,"Nueva Orleans, Lousiana, USA",https://es.wikipedia.org/wiki/UFC_16,Not Found,Not Found
19,UFC 17: Redemption,15 de mayo de 1998,Mobile Civic Center,"Mobile, Alabama, USA",https://es.wikipedia.org/wiki/UFC_17,Not Found,Not Found
20,UFC Ultimate Brazil,16 de octubre de 1998,Ginásio da Portuguesa,"São Paulo, Brasil",https://es.wikipedia.org/wiki/UFC_Brazil,Not Found,Not Found
21,UFC 18: The Road to the Heavyweight Title,8 de enero de 1999,Pontchartrain Center,"Nueva Orleans, Lousiana, USA",https://es.wikipedia.org/wiki/UFC_18,Not Found,Not Found
22,UFC 19: Ultimate Young Guns,5 de marzo de 1999,Casino Magic Bay St. Louis,"Bay St. Louis, Mississippi, USA",https://es.wikipedia.org/wiki/UFC_19,Not Found,Not Found
23,UFC 20: Battle for the Gold,7 de mayo de 1999,Boutwell Auditorium,"Birmingham, Alabama, USA",https://es.wikipedia.org/wiki/UFC_20,Not Found,Not Found


In [25]:
link = 'https://es.wikipedia.org/wiki/UFC_269'


In [26]:
response = requests.get(link)
soup = BS(response.text)
scrap_table = soup.find('tbody')
rows = scrap_table.find_all('tr')
elementos = [row.text.split('\n') for row in rows]
print(elementos)
for elemento in elementos:
    if 'Recaudación' in elemento:
        print(elemento[1])
        break
    if elemento == elementos[-1]:
        print('Not Found')

[['UFC 269: Oliveira vs. Poirier'], ['Información'], ['Promoción', 'Ultimate Fighting Championship'], ['Fecha', '11 de diciembre de 2021'], ['Sede', 'T-Mobile Arena'], ['Ciudad', ' Paradise, Nevada, Estados Unidos'], ['Asistencia', '18471[1]\u200b'], ['Recaudación', '$8,000,000[1]\u200b'], ['Cronología de eventos'], ['', '', '', 'UFC on ESPN: Font vs. Aldo', '', 'UFC 269: Oliveira vs. Poirier', '', 'UFC Fight Night: Lewis vs. Daukaus', ''], ['', 'UFC on ESPN: Font vs. Aldo', '', 'UFC 269: Oliveira vs. Poirier', '', 'UFC Fight Night: Lewis vs. Daukaus', ''], ['[editar datos en Wikidata]']]
$8,000,000[1]​
